In [2]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plt

import os
import numpy as np
from random import shuffle
import copy
import time

In [3]:
# Check availability of GPU
use_gpu = torch.cuda.is_available()

In [4]:
train_path = "C:\\Users\\smitr\\Desktop\\ucf10_resnetFeat\\train"
test_path  = "C:\\Users\\smitr\\Desktop\\ucf10_resnetFeat\\test"

In [5]:
classes = os.listdir(train_path)
classes.sort()
labels = np.arange(10)  # For 10 classes
trainShuffList = []
labelShuffList = []

for c in range(10):
    files = os.listdir(train_path+"\\"+classes[c])
    for f in files:
        trainShuffList.append(classes[c]+"\\"+f)
        labelShuffList.append(float(labels[c]))
# Shuffling data list and label list
trainList = list(zip(trainShuffList, labelShuffList))
shuffle(trainList)
trainShuffList, labelShuffList = zip(*trainList)        

In [6]:
trainShuffList[:10]

('BaseballPitch\\v_BaseballPitch_g14_c02.pt',
 'ApplyEyeMakeup\\v_ApplyEyeMakeup_g02_c02.pt',
 'BandMarching\\v_BandMarching_g12_c05.pt',
 'BasketballDunk\\v_BasketballDunk_g01_c06.pt',
 'ApplyLipstick\\v_ApplyLipstick_g17_c02.pt',
 'BaseballPitch\\v_BaseballPitch_g06_c03.pt',
 'Basketball\\v_Basketball_g03_c03.pt',
 'Archery\\v_Archery_g05_c04.pt',
 'Archery\\v_Archery_g06_c02.pt',
 'BaseballPitch\\v_BaseballPitch_g19_c01.pt')

In [7]:
# Test list for loading feature tensors
testList = []
testLabelList = []

for c in range(10):
    files = os.listdir(test_path+"\\"+classes[c])
    for f in files:
        testList.append(classes[c]+"\\"+f)
        testLabelList.append(float(labels[c]))

<h2> Define LSTM architecture </h2>

In [8]:
class net_LSTM(nn.Module):
    def __init__(self, input_sz, hidden_sz, nLayers, nClasses):
        super(net_LSTM, self).__init__()
        self.lstm = nn.LSTM(input_sz, hidden_sz, nLayers, batch_first=True)
        self.fc = nn.Linear(hidden_sz, nClasses)
        
    def forward(self, x):
        out, _ = self.lstm(x)
        # Output of hidden state from last time step
        out = self.fc(out[:,-1,:])
        return out

<h2> Define train routine </h2>

In [9]:
def train(net, inputs, labels, optimizer, criterion):
    net.train(True)
    if use_gpu:
        inputs = Variable(inputs.cuda())
        labels = Variable(labels.cuda())
    else:
        inputs = Variable(inputs)
        labels = Variable(labels)
    
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)
    # Initialize gradients to zero
    optimizer.zero_grad()
    # Compute error
    loss = criterion(F.log_softmax(outputs), labels)
    # Backprop
    loss.backward()
    # Update params
    optimizer.step()
    if use_gpu:
        correct = (predicted.cpu() == labels.data.cpu()).sum().item()
    else:
        correct = (predicted == labels.data).sum().item()
    return net, loss.data.item(), correct

<h2> Define test routine </h2>

In [10]:
def test(net, inputs, labels, criterion):
    net.train(False)
    if use_gpu:
        inputs = Variable(inputs.cuda())
        labels = Variable(labels.cuda())
    else:
        inputs, labels = Variable(inputs), Variable(labels)
    
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)
    # Compute loss 
    loss = criterion(F.log_softmax(outputs), labels)
    if use_gpu:
        correct = (predicted.cpu() == labels.data.cpu()).sum().item()
    else:
        correct = (predicted == labels.data).sum().item()
    return loss.data.item(), correct

<h2> Initialize the network </h2>

In [21]:
net = net_LSTM(512, 256, 2, 10)
if use_gpu:
    net = net.cuda()

<h2> Define loss function & optimizer </h2>

In [22]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-4)

<h2> Training the network </h2>

In [23]:
epochs = 3000
bSize = 32 # Batch size
L = 32 # Number of time steps

bCount = len(trainShuffList)//bSize # Number of batches in train set
lastBatch = len(trainShuffList)%bSize # Number of samples in last batch of train set

test_bCount = len(testList)//bSize # Number of batches in test set
test_lastBatch = len(testList)%bSize # Number of samples in last batch of test set

# Lists for saving train/test loss and accuracy
trainLoss = []
trainAcc = []
testLoss = []
testAcc = []

start = time.time()

for epochNum in range(epochs):
    # Shuffling train data for each epoch
    trainList = list(zip(trainShuffList, labelShuffList))
    shuffle(trainList)
    trainShuffList, labelShuffList = zip(*trainList)
    
    trainRunLoss = 0.0
    testRunLoss = 0.0
    trainRunCorr = 0
    testRunCorr = 0
    
    epochStart = time.time()
    
    ## Train
    # Load data tensors batchwise     
    idx = 0    
    for bNum in range(bCount):
        first = True
        # Loading one batch
        for dNum in range(idx,idx+bSize):
            if first:
                loadData = torch.load(train_path+"\\"+trainShuffList[dNum])
                sz = loadData.size(0)   # No.of images extracted from video
                idx1 = torch.from_numpy(np.arange(0,(sz//L)*L,sz//L))
                batchData = torch.index_select(loadData,dim=0,index=idx1.long()).unsqueeze(0)
                batchLabel = torch.Tensor([labelShuffList[dNum]]).long()                          
                first = False                
            else:
                loadData = torch.load(train_path+"\\"+trainShuffList[dNum])
                sz = loadData.size(0)
                idx1 = torch.from_numpy(np.arange(0,(sz//L)*L,sz//L))
                tempData = torch.index_select(loadData,dim=0,index=idx1.long()).unsqueeze(0)
                batchData = torch.cat((batchData,tempData), dim=0)
                batchLabel = torch.cat((batchLabel,torch.Tensor([labelShuffList[dNum]]).long()),dim=0)            
        
        # Train the network on current batch
        net, tr_loss, tr_corr = train(net, batchData, batchLabel, optimizer, criterion)
        trainRunLoss += tr_loss
        trainRunCorr += tr_corr
        idx += bSize
        
    # Loading last batch
    if lastBatch != 0:        
        first = True
        for dNum in range(idx,idx+lastBatch):
            if first:
                loadData = torch.load(train_path+"\\"+trainShuffList[dNum])
                sz = loadData.size(0)
                idx1 = torch.from_numpy(np.arange(0,(sz//L)*L,sz//L))
                batchData = torch.index_select(loadData,dim=0,index=idx1.long()).unsqueeze(0)
                batchLabel = torch.Tensor([labelShuffList[dNum]]).long()
                first = False                
            else:
                loadData = torch.load(train_path+"\\"+trainShuffList[dNum])
                sz = loadData.size(0)
                idx1 = torch.from_numpy(np.arange(0,(sz//L)*L,sz//L))
                tempData = torch.index_select(loadData,dim=0,index=idx1.long()).unsqueeze(0)
                batchData = torch.cat((batchData,tempData), dim=0)
                batchLabel = torch.cat((batchLabel,torch.Tensor([labelShuffList[dNum]]).long()),dim=0)          
        
        # Training network on last batch
        net, tr_loss, tr_corr = train(net, batchData, batchLabel, optimizer, criterion)
        trainRunLoss += tr_loss
        trainRunCorr += tr_corr
    
    # Average training loss and accuracy for each epoch
    avgTrainLoss = trainRunLoss/float(len(trainShuffList))
    trainLoss.append(avgTrainLoss)
    avgTrainAcc = trainRunCorr/float(len(trainShuffList))
    trainAcc.append(avgTrainAcc)
    
    ## Test
    # Load data tensors batchwise     
    idx = 0    
    for bNum in range(test_bCount):
        first = True
        # Loading one batch
        for dNum in range(idx,idx+bSize): 
            if first:
                loadData = torch.load(test_path+"\\"+testList[dNum])
                sz = loadData.size(0)
                idx1 = torch.from_numpy(np.arange(0,(sz//L)*L,sz//L))
                batchData = torch.index_select(loadData,dim=0,index=idx1.long()).unsqueeze(0)
                batchLabel = torch.Tensor([testLabelList[dNum]]).long()
                first = False                
            else:
                loadData = torch.load(test_path+"\\"+testList[dNum])
                sz = loadData.size(0)
                idx1 = torch.from_numpy(np.arange(0,(sz//L)*L,sz//L))
                tempData = torch.index_select(loadData,dim=0,index=idx1.long()).unsqueeze(0)
                batchData = torch.cat((batchData,tempData), dim=0)
                batchLabel = torch.cat((batchLabel,torch.Tensor([testLabelList[dNum]]).long()),dim=0)            
        
        # Test the network on current batch
        ts_loss, ts_corr = test(net, batchData, batchLabel, criterion)
        testRunLoss += ts_loss
        testRunCorr += ts_corr
        idx += bSize
     
    # Loading last batch    
    if test_lastBatch != 0:        
        first = True
        for dNum in range(idx,idx+test_lastBatch):
            if first:
                loadData = torch.load(test_path+"\\"+testList[dNum])
                sz = loadData.size(0)
                idx1 = torch.from_numpy(np.arange(0,(sz//L)*L,sz//L))
                batchData = torch.index_select(loadData,dim=0,index=idx1.long()).unsqueeze(0)               
                batchLabel = torch.Tensor([testLabelList[dNum]]).long()
                first = False                
            else:
                loadData = torch.load(test_path+"\\"+testList[dNum])
                sz = loadData.size(0)
                idx1 = torch.from_numpy(np.arange(0,(sz//L)*L,sz//L))
                tempData = torch.index_select(loadData,dim=0,index=idx1.long()).unsqueeze(0)
                batchData = torch.cat((batchData,tempData), dim=0)
                batchLabel = torch.cat((batchLabel,torch.Tensor([testLabelList[dNum]]).long()),dim=0)          
        
        # Test network on last batch
        ts_loss, ts_corr = test(net, batchData, batchLabel, criterion)
        testRunLoss += ts_loss
        testRunCorr += tr_corr
        
    # Average testing loss and accuracy for each epoch
    avgTestLoss = testRunLoss/float(len(testList))
    testLoss.append(avgTestLoss)
    avgTestAcc = testRunCorr/float(len(testList))
    testAcc.append(avgTestAcc)   
    
    
    if (epochNum+1) % 200 == 0:
        epochEnd = time.time()-epochStart
        print('Iteration: {:.0f} /{:.0f};  Training Loss: {:.6f} ; Training Acc: {:.3f}'\
              .format(epochNum + 1,epochs, avgTrainLoss, avgTrainAcc*100))
        print('Iteration: {:.0f} /{:.0f};  Testing Loss: {:.6f} ; Testing Acc: {:.3f}'\
              .format(epochNum + 1,epochs, avgTestLoss, avgTestAcc*100))
    
        print('Time consumed: {:.0f}m {:.0f}s'.format(epochEnd//60,epochEnd%60))        
end = time.time()-start
print('Training completed in {:.0f}m {:.0f}s'.format(end//60,end%60))      
        

c:\users\smitr\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app
c:\users\smitr\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


Iteration: 200 /3000;  Training Loss: 0.073188 ; Training Acc: 12.397
Iteration: 200 /3000;  Testing Loss: 0.074294 ; Testing Acc: 3.971
Time consumed: 0m 2s
Iteration: 400 /3000;  Training Loss: 0.073247 ; Training Acc: 11.304
Iteration: 400 /3000;  Testing Loss: 0.074433 ; Testing Acc: 11.552
Time consumed: 0m 2s
Iteration: 600 /3000;  Training Loss: 0.073095 ; Training Acc: 11.668
Iteration: 600 /3000;  Testing Loss: 0.074260 ; Testing Acc: 4.693
Time consumed: 0m 2s
Iteration: 800 /3000;  Training Loss: 0.073245 ; Training Acc: 10.939
Iteration: 800 /3000;  Testing Loss: 0.074374 ; Testing Acc: 10.830
Time consumed: 0m 2s
Iteration: 1000 /3000;  Training Loss: 0.071666 ; Training Acc: 13.856
Iteration: 1000 /3000;  Testing Loss: 0.074342 ; Testing Acc: 10.830
Time consumed: 0m 2s
Iteration: 1200 /3000;  Training Loss: 0.038205 ; Training Acc: 49.954
Iteration: 1200 /3000;  Testing Loss: 0.046877 ; Testing Acc: 39.350
Time consumed: 0m 2s
Iteration: 1400 /3000;  Training Loss: 0.041

In [ ]:
# Plotting training loss vs Epochs
fig1 = plt.figure(1)        
plt.plot(range(epochs),trainLoss,'r-',label='train')  
plt.plot(range(epochs),testLoss,'g-',label='test') 
plt.legend(loc='upper left')
plt.xlabel('Epochs')
plt.ylabel('Loss')   

# Plotting testing accuracy vs Epochs
fig2 = plt.figure(2)        
plt.plot(range(epochs),trainAcc,'r-',label='train')    
plt.plot(range(epochs),testAcc,'g-',label='test')        
plt.legend(loc='upper left')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')